In [ ]:
import numpy as np

from steenroder import *
import gudhi

from matplotlib import pyplot as plt

from scipy.io import loadmat

from sklearn.manifold import Isomap
from hdbscan import HDBSCAN

from gtda.homology import VietorisRipsPersistence
from gtda.plotting import plot_point_cloud

This dataset of 6040 samples from the configuration space of the cyclo-octane molecule $C_{8} H_{16}$ is described in https://www.frontiersin.org/articles/10.3389/frai.2021.668302/full#B48 as follows:

> What do we mean by “global shape”? Consider, for example, conformations of the cyclo-octane molecule $C_8 H_{16}$, which consists of a ring of eight carbons atoms, each bonded to a pair of hydrogen atoms (see Figure 4, left). The locations of the carbon atoms in a conformation approximately determine the locations of the hydrogen atoms via energy minimization, and hence each molecule conformation can be mapped to a point in $\mathbb{R}^{24} = \mathbb{R}^{8 \cdot 3}$, as the location of each carbon atom can be specified by three coordinates. This map realizes the conformation space of cyclo-octane as a subset of $\mathbb{R}^{24}$, and then we mod out by rigid rotations and translations. Topologically, the conformation space of cyclo-octane turns out to be the union of a sphere with a Klein bottle, glued together along two circles of singularities (see Figure 4, right). This model was obtained by Martin et al. (2010), Martin and Watson (2011), and Brown et al. (2008), who furthermore obtain a triangulation of this dataset (a representation of the dataset as a union of vertices, edges, and triangles).

In [ ]:
cyclo_octane = loadmat("../data/pointsCycloOctane.mat")['pointsCycloOctane']
cyclo_octane.shape

The circles of singularities can be found by several methods. Traditionally, local PCA was used. Here, we use the singular set found by using the methods in https://www.pnas.org/content/117/33/19664:

In [ ]:
singular_indices = loadmat("../data/singularity_indicesCycloOctane_PH0.5.mat")['singularity_indices_PH'].flatten() - 1
nonsingular_indices = np.array([x for x in range(len(cyclo_octane)) if x not in singular_indices])
print(f"{len(singular_indices)} points in the singular set")

Let us store the non-singular portion for later use

In [ ]:
cyclo_octane_nonsingular = cyclo_octane[nonsingular_indices]

A dimensionality reduction algorithm such as ISOMAP can help us visualize the full dataset in 3D:

In [ ]:
isomap = Isomap(n_components=3).fit_transform(cyclo_octane)

In [ ]:
plot_point_cloud(isomap, plotly_params={"trace": {"marker_size": 1},
                                        "layout": {"height": 600, "width": 600}})

We can also visualize the singular and non-singular portions:

In [ ]:
isomap_nonsingular = isomap[nonsingular_indices]
isomap_singular = isomap[singular_indices]

In [ ]:
plot_point_cloud(isomap_nonsingular, plotly_params={"trace": {"marker_size": 1},
                                                    "layout": {"height": 600, "width": 600}})

We can see that the singular set does look like two circles in the ISOMAP projection:

In [ ]:
plot_point_cloud(isomap_singular, plotly_params={"trace": {"marker_size": 1},
                                                 "layout": {"height": 600, "width": 600}})

From previous literature, we expect the nonsingular part of the data to be the union of a 2-sphere and a Klein bottle. We partition the non-singular part using the clustering algorithm HDBSCAN:

In [ ]:
HD = HDBSCAN(min_samples=2, min_cluster_size=300, alpha=1., cluster_selection_epsilon=0)
HD.fit(cyclo_octane_nonsingular)

With these hyperparameters, the portion we are interested in turns out to be the cluster labelled as `3` by HDBSCAN:

In [ ]:
mask_klein = HD.labels_ == 3
isomap_klein = isomap_nonsingular[mask_klein]

plot_point_cloud(isomap_klein, plotly_params={"trace": {"marker_size": 1},
                                              "layout": {"height": 600, "width": 600}})

In [ ]:
mask_sphere = np.isin(HD.labels_, [0, 1, 2])
isomap_sphere = isomap_nonsingular[mask_sphere]

plot_point_cloud(isomap_sphere, plotly_params={"trace": {"marker_size": 1},
                                               "layout": {"height": 600, "width": 600}})

# Steenrod barcode

In [ ]:
cyclo_octane_klein = cyclo_octane_nonsingular[mask_klein]
cyclo_octane_sphere = cyclo_octane_nonsingular[mask_sphere]

VietorisRipsPersistence(max_edge_length=1.5, infinity_values=np.inf).fit_transform_plot(
    [cyclo_octane_klein],
    plotly_params={"traces": {"marker_size": 2},
                   "layout": {"title": "Klein bottle component"}}
);
VietorisRipsPersistence(max_edge_length=1.5, infinity_values=np.inf).fit_transform_plot(
    [cyclo_octane_sphere],
    plotly_params={"traces": {"marker_size": 2},
                   "layout": {"title": "Sphere component"}}
);

We construct a Rips filtration of the selected dataset, allegedly a Klein bottle.

In [ ]:
max_edge_length = 1.2

In [ ]:
simplex_tree = gudhi.RipsComplex(points=cyclo_octane_klein, max_edge_length=max_edge_length).\
    create_simplex_tree(max_dimension=1)

for i, _ in enumerate(simplex_tree.get_filtration()):
        pass

len_filtration = i + 1
print(f"Filtration with {len_filtration} simplices up to dimension 1 initially")

This filtration is too large to be handled by Steenroder in a reasonable amount of time when expanded all the way to dimension 3, so we apply edge collapses iteratively. *Caution: this step may take a while to run*

In [ ]:
while True:
    simplex_tree.collapse_edges()

    for i, _ in enumerate(simplex_tree.get_filtration()):
        pass

    if i + 1 == len_filtration:
        break
    else:
        len_filtration = i + 1
        print(f"{len_filtration} simplices up to dimension 1")

We now construct the full Rips filtration up to dimension-3 simplices, and run Steenroder:

In [ ]:
simplex_tree.expansion(3)  # Get the three-simplices after collapse

filtration = []
filtration_values = []
for s in simplex_tree.get_filtration():
    filtration.append(tuple(s[0]))
    filtration_values.append(s[1])

filtration_values = np.asarray(filtration_values, dtype=np.float64)

print(f"Filtration with {len(filtration)} simplices up to dimension 3")

In [ ]:
k = 1

barcode, st_barcode = barcodes(
    k,
    filtration,
    homology=True,
    filtration_values=filtration_values,
    return_filtration_values=True,
    verbose=True
    )

st_barcode